In [14]:
# imports
import torch
from torch.utils.data import DataLoader
from torch import nn

# for built in datasets
import torchvision
from torchvision import transforms, datasets

# import my_datasets
# import importlib
# importlib.reload(module=my_datasets)

<module 'my_datasets' from '/Users/rachellelang/Documents/College/Learning from Topology URS Research/my_datasets.py'>

In [16]:
# variables
D = 2 # dimensions
POINTS = 100 # number of data points
BATCH_SIZE = 10 # for stochastic gradient descent
EPOCHS = 3 # not exactly sure what this is for yet

In [44]:
# download MNIST data
train = datasets.MNIST('', train=True, download=True, transform = transforms.Compose([transforms.ToTensor()]))
test = datasets.MNIST('', train=False, download=True, transform = transforms.Compose([transforms.ToTensor()]))

In [46]:
# now for the data loader
# set = my_datasets.CircleDataset(POINTS)
loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)

In [40]:
# nueral network class
class NeuralNetwork(nn.Module):

    def __init__(self, i, h, o): # where i is the input dimensions, h is neurons in the hidden layer, o is the number of categories
        super().__init__()
        # these are actually functions
        self.layer1 = nn.Linear(i, h) # input, output
        self.layer2 = nn.Linear(h, h)
        self.layer3 = nn.Linear(h, o)

    def forward(self, x):
        # seems like linear would be doing the weights and biases and then relu is activation function
        x = nn.functional.relu(self.layer1(x))
        x = nn.functional.relu(self.layer2(x))
        x = self.layer3(x)
        # don't relu last layer because we do softmax instead which gives probability distribution
        return nn.functional.log_softmax(x, dim=1) # dim is dimension that needs to sum to one (dim 0 is across batches)

In [41]:
# implementing neural network

net = NeuralNetwork(28*28, 64, 10)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

for epoch in range(EPOCHS):
    for batch in loader:
        x, y = batch
        net.zero_grad() # I don't really get why you need this
        output = net(x.view(-1, 28*28)) # seems to work with dimensions but maybe it's not apparently calling net is same as calling forward?
        loss = nn.functional.nll_loss(output, y) # the loss function for scalar values instead of one hot (vector with one 1 and rest 0s)
        loss.backward() # magic
        optimizer.step() # adjusts weights
    print(loss)


tensor(0.0287, grad_fn=<NllLossBackward0>)
tensor(0.0096, grad_fn=<NllLossBackward0>)
tensor(0.0236, grad_fn=<NllLossBackward0>)


In [47]:
# evaluate the network

correct = 0
total = 0

with torch.no_grad(): # tells pytorch your not calculating gradients so reduces memory consumption
    for data in test_loader:
        x, y = data
        output = net(x.view(-1, 784))
        for idx, i in enumerate(output): # enumerate lets you go over something by index and by actual object
            if torch.argmax(i) == y[idx]: # torch.argmax returns the index of the max value in the tensor
                correct += 1
            total += 1

print('Accuracy: ', round(correct/total, 3))



Accuracy:  0.969


To Do
• cross entropy vs nll_loss
• use gpus
• test on real datasets first, then test on manifolds
• look at math lecture slides
• probably read Chicago paper - look at number of layers they use and see if Google Colab can handle it
• read ising model, renormalization group on wikipedia